# DSMCER HW 4

###  Please note carefully:
- Any plots should be the highest quality possible and, in your judgement, ready for inclusion in a formal report or publication. This includes labeling, font sizes, etc.

- While we encourage you to chat with eachother and work together, you should not be copying code.

- Remember one of the biggest advantages of the python data stack - someone has probably written code for what you need to do. If you find yourself writing lines and lines of complex code here there is probably a better way, eg. a function written in numpy, scipy, pandas, seaborn etc.

***

## Part 0 - Preparation (1 pt)

- Load the HCEPDB dataset, randomly select 500,000 datapoints to act as your dataset
- Your goal from this notebook will be to predict energy gap of a material __without having to synthesize it__. Drop the columns in the data you will not need.
- Use the provided code to extract features for each molecule based on its smiles string alone. Notice that I have made the function raise an error when feature extraction fails. What will you do in this case? Handle this scenario however you deem suitable. Feel free to modify the code given if you would like, but it would be most pythonic to handle the error outside of the function.
- Split the data randomly into a development set and a test set.

_Note_: extracting features for all 500k examples can take ~15 minutes. It may be worth saving the results of Part 0 to file so that never have to run it again eg. if the kernel restarts.

In [1]:
import numpy as np
import pandas as pd

import rdkit.Chem
import rdkit.Chem.Descriptors
import rdkit.Chem.rdMolDescriptors
import rdkit.ML.Descriptors.MoleculeDescriptors

### Provided Code:

In [2]:
features_to_extract = ['FractionCSP3', 'HeavyAtomCount', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'MolLogP', 'MolMR']
feature_calculator = rdkit.ML.Descriptors.MoleculeDescriptors.MolecularDescriptorCalculator(features_to_extract)

class BadSmilesException(Exception):
    """Indicates SMILES string not valid for desired features."""
    def __init__(self, smiles, *args):
        super().__init__(args)
        self.smiles = smiles

    def __str__(self):
        return f'SMILES: "{self.smiles}" invalid, could not compute features.'

def extract_features_for_smiles(smiles_string: str):
    """Compute a feature vector for a molecule based on its molecular graph and rdkit.
    
    Parameters
    ----------
    smiles_string: str
        Smiles connectivity of molecule
        
    Returns
    -------
    numpy row vector of featutes
    
    Exceptions
    ----------
    BadSmilesException : cannot compute features for this type of molecule
    """
    if not type(smiles_string) == str:
        raise ValueError(f"Expecting `smiles_string` to be a string type, not {type(smiles_string)}")
    else:
        pass
    
    mol = rdkit.Chem.MolFromSmiles(smiles_string)
    descriptors = feature_calculator.CalcDescriptors(mol)
    descriptors = np.array(descriptors).reshape(1,-1)
    
    if np.any(np.isclose(descriptors, -666)):
        raise BadSmilesException(smiles_string)
    else:
        pass
    return descriptors

### Do part 0 here:

***

## Part 1 - Comment on feature extraction (1 pt)

Feature engineering is composed of both extraction and selection. You had a little bit of practice on extraction in HW2, so I have done some extraction for you here using `rdkit`. You will need to do some selection in the rest of the notebook, but before that, an aside:

__Write a few sentences__ describing how you might go about extracting features for a situtuation where your examples are something __other than a molecules__. What would some of your features be? Choose from one of the following:

- Spectra from an instrument
- Heat exchangers
- Cells

***

## Part 2 - Feature selection (2 pt)

#### a. From the features provided, arrive at a final set of features to give a ML model using feature selection techniques such as filtering, forward selection, etc. __Do not forget standardization__, without which most methods will be invalidated. 
#### b. Write a few sentences describing your choices. 

***

## Part 3 - Architecture optimization (3 pt)

#### a. Make an informed decision about a machine learning architecture to use to predict the energy gap. 
  - You may run some code to help you make this choice if you would like.
  - You may also inform your decision by known aspects of the architecture and how that may or may not work with your system. 

One or both is sufficient so long as it is well thought out. Write a few sentences describing your process and which architecture you chose

#### b. Choose __one__ hyperparameter associated with the chosen architecture to optimize. Use a validation scheme of your choice to determine what value the hyperparameter should be.

Write a few senteces describing your process and why you made those choices. Report the approximate carbon cost of finding the best hyperparameter.

#### c. With the chosen architecture and the identified best value of the hyperparameter, __train a new model on the entire development set__. Make prediction on the test set, and report its performance, and create a partity plot that shows the regression predictions.